In [3]:
import json
import os
import numpy as np

def extract_features(file_path, result_folder):
    """
    Extract features and save plots to respective folders.
    """
    # 결과 폴더 생성
    os.makedirs(result_folder, exist_ok=True)
    
    # 파일 이름 추출
    file_name = os.path.splitext(os.path.basename(file_path))[0]

    # 각 특징별 폴더 생성 및 이미지 경로 설정
    def get_image_path(feature_name):
        feature_folder = os.path.join(result_folder, feature_name)
        os.makedirs(feature_folder, exist_ok=True)
        return os.path.join(feature_folder, f"{file_name}.png")

    # 예시 특징 데이터
    return {
        "file_path": file_path,
        "features": {
            "zero_crossing_rate": 0.012,
            "harmonic_to_noise_ratio": 0.75,
            "spectral_flatness": 0.3,
            "spectral_rolloff": 2800.0,
            "rms_energy": 0.05,
            "entropy": 1.45,
            "waveform": get_image_path("waveform"),
            "envelope": get_image_path("envelope"),
            "cepstrum": get_image_path("cepstrum"),
            "mel_spectrogram_with_axes": get_image_path("mel_spectrogram_with_axes"),
            "mel_spectrogram_no_axes": get_image_path("mel_spectrogram_no_axes"),
            "mfcc": get_image_path("mfcc"),
            "linear_spectrogram_with_axes": get_image_path("linear_spectrogram_with_axes"),
            "linear_spectrogram_no_axes": get_image_path("linear_spectrogram_no_axes"),
            "chroma_features": get_image_path("chroma_features"),
            "spectral_centroid": get_image_path("spectral_centroid"),
            "spectral_bandwidth": get_image_path("spectral_bandwidth"),
            "power_spectrum": get_image_path("power_spectrum"),
            "std": 0.123,
            "avg": 0.456
        },
    }

def create_json_structure(data_entries, output_json_path):
    """
    Create a JSON file with structured data entries.
    """
    data = {"audio_features": data_entries}
    with open(output_json_path, 'w') as json_file:
        json.dump(data, json_file, indent=4)

def main():
    """
    Main function to process all files and extract features.
    """
    classes = ["ToyCar", "ToyTrain", "bearing", "fan", "gearbox", "slider", "valve"]
    data_type = ['test', 'train']
    all_entries = []

    for class_name in classes:
        for data in data_type:
            folder_path = f'../unziped/dev/{class_name}/{data}'
            result_folder = os.path.join(folder_path, "Feature_Extraction_Results")
            os.makedirs(result_folder, exist_ok=True)
            
            # 모든 wav 파일 처리
            wav_files = [f for f in os.listdir(folder_path) if f.endswith('.wav')]
            
            for wav_file in wav_files:
                file_path = os.path.join(folder_path, wav_file)
                features = extract_features(file_path, result_folder)
                
                # 섹션 정보 추출 (파일명에서 'section_' 뒤의 두 글자)
                if 'section_' in wav_file:
                    section_start = wav_file.find('section_') + len('section_')
                    section = wav_file[section_start:section_start+2]
                else:
                    section = '00'  # 기본값

                # note 추가 (자동 생성)
                note = f"This is a {class_name} machine, data type: {data}, file: {wav_file}"
                
                entry = {
                    "file_path": features["file_path"],
                    "machine_type": class_name,
                    "section": section,
                    "domain": data,  # 'train' 또는 'test'
                    "features": features["features"],
                    "note": note  # 주석 추가
                }
                all_entries.append(entry)
    
    # JSON 파일로 저장
    output_json_path = 'audio_features.json'
    create_json_structure(all_entries, output_json_path)
    print(f"JSON 파일이 '{output_json_path}' 경로에 저장되었습니다.")

if __name__ == "__main__":
    main()


JSON 파일이 'audio_features.json' 경로에 저장되었습니다.


In [ ]:
import json
import os
import sys
import numpy as np
import librosa
import librosa.display
import matplotlib
import matplotlib.pyplot as plt
from multiprocessing import Pool, cpu_count

# Matplotlib 백엔드를 'Agg'로 설정 (비대화형 백엔드)
matplotlib.use('Agg')

def extract_features(file_path, result_folder, min_freq=20, max_freq=8000, normalization='min-max'):
    """
    Extract features from an audio file and save corresponding plots.

    Parameters:
        file_path (str): Path to the audio file.
        result_folder (str): Directory to save feature plots.
        min_freq (float): Minimum frequency for plotting.
        max_freq (float): Maximum frequency for plotting.
        normalization (str): Type of normalization for MFCC.

    Returns:
        dict: Dictionary containing file path and extracted features.
    """
    try:
        # 오디오 로드
        y, sr = librosa.load(file_path, sr=None)
    except Exception as e:
        print(f"오디오 파일을 로드하는 중 오류 발생: {file_path}\n오류 내용: {e}", file=sys.stderr)
        return None

    # 특징 딕셔너리 초기화
    features = {}

    try:
        # 1. Zero Crossing Rate
        zero_crossing_rate = np.mean(librosa.feature.zero_crossing_rate(y=y))
        features['zero_crossing_rate'] = zero_crossing_rate

        # 2. Harmonic to Noise Ratio (HNR)
        y_harmonic, y_percussive = librosa.effects.hpss(y)
        hnr = np.mean(np.abs(y_harmonic)) / (np.mean(np.abs(y_percussive)) + 1e-10)
        features['harmonic_to_noise_ratio'] = hnr

        # 3. Spectral Flatness
        spectral_flatness = np.mean(librosa.feature.spectral_flatness(y=y))
        features['spectral_flatness'] = spectral_flatness

        # 4. Spectral Rolloff
        spectral_rolloff = np.mean(librosa.feature.spectral_rolloff(y=y, sr=sr))
        features['spectral_rolloff'] = spectral_rolloff

        # 5. RMS Energy
        rms_energy = np.mean(librosa.feature.rms(y=y))
        features['rms_energy'] = rms_energy

        # 6. Entropy (Shannon Entropy)
        histogram, bin_edges = np.histogram(y, bins=256, density=True)
        entropy = -np.sum(histogram * np.log2(histogram + 1e-10))
        features['entropy'] = entropy

        # 7. Standard Deviation and Average
        features['std'] = np.std(y)
        features['avg'] = np.mean(y)

        # 결과 폴더 생성
        os.makedirs(result_folder, exist_ok=True)

        # 파일 이름 추출
        file_name = os.path.splitext(os.path.basename(file_path))[0]

        # 각 특징별로 폴더 생성 및 이미지 저장
        def save_plot(fig, feature_name):
            feature_folder = os.path.join(result_folder, feature_name)
            os.makedirs(feature_folder, exist_ok=True)
            return os.path.join(feature_folder, f"{file_name}.png")

        # 8. Waveform Plot
        fig, ax = compute_and_plot_waveform(y, sr)
        waveform_path = save_plot(fig, 'waveform')
        fig.savefig(waveform_path, bbox_inches='tight', pad_inches=0)
        plt.close(fig)
        features['waveform'] = waveform_path

        # 9. Envelope Plot
        fig, ax = compute_and_plot_envelope(y, sr)
        envelope_path = save_plot(fig, 'envelope')
        fig.savefig(envelope_path, bbox_inches='tight', pad_inches=0)
        plt.close(fig)
        features['envelope'] = envelope_path

        # 10. Cepstrum Plot
        fig, ax = compute_and_plot_cepstrum(y, sr, min_freq=min_freq, max_freq=max_freq)
        cepstrum_path = save_plot(fig, 'cepstrum')
        fig.savefig(cepstrum_path, bbox_inches='tight', pad_inches=0)
        plt.close(fig)
        features['cepstrum'] = cepstrum_path

        # 11. Mel Spectrogram Plot
        # (a) With axes
        fig, ax = compute_and_plot_mel_spectrogram(y, sr, display_axes=True)
        mel_spectrogram_with_axes_path = save_plot(fig, 'mel_spectrogram_with_axes')
        fig.savefig(mel_spectrogram_with_axes_path, bbox_inches='tight', pad_inches=0)
        plt.close(fig)
        features['mel_spectrogram_with_axes'] = mel_spectrogram_with_axes_path

        # (b) Without axes
        fig, ax = compute_and_plot_mel_spectrogram(y, sr, display_axes=False)
        mel_spectrogram_no_axes_path = save_plot(fig, 'mel_spectrogram_no_axes')
        fig.savefig(mel_spectrogram_no_axes_path, bbox_inches='tight', pad_inches=0)
        plt.close(fig)
        features['mel_spectrogram_no_axes'] = mel_spectrogram_no_axes_path

        # 12. MFCC Plot
        fig, ax = compute_and_plot_mfcc(y, sr, normalization=normalization)
        mfcc_path = save_plot(fig, 'mfcc')
        fig.savefig(mfcc_path, bbox_inches='tight', pad_inches=0)
        plt.close(fig)
        features['mfcc'] = mfcc_path

        # 13. Linear Spectrogram Plot
        # (a) With axes
        fig, ax = compute_and_plot_linear_spectrogram(y, sr, n_fft=256, hop_length=128, display_axes=True)
        linear_spectrogram_with_axes_path = save_plot(fig, 'linear_spectrogram_with_axes')
        fig.savefig(linear_spectrogram_with_axes_path, bbox_inches='tight', pad_inches=0)
        plt.close(fig)
        features['linear_spectrogram_with_axes'] = linear_spectrogram_with_axes_path

        # (b) Without axes
        fig, ax = compute_and_plot_linear_spectrogram(y, sr, n_fft=256, hop_length=128, display_axes=False)
        linear_spectrogram_no_axes_path = save_plot(fig, 'linear_spectrogram_no_axes')
        fig.savefig(linear_spectrogram_no_axes_path, bbox_inches='tight', pad_inches=0)
        plt.close(fig)
        features['linear_spectrogram_no_axes'] = linear_spectrogram_no_axes_path

        # 14. Chroma Features Plot
        fig, ax = compute_and_plot_chroma_features(y, sr)
        chroma_features_path = save_plot(fig, 'chroma_features')
        fig.savefig(chroma_features_path, bbox_inches='tight', pad_inches=0)
        plt.close(fig)
        features['chroma_features'] = chroma_features_path

        # 15. Spectral Centroid Plot
        fig, ax = compute_and_plot_spectral_centroid(y, sr)
        spectral_centroid_path = save_plot(fig, 'spectral_centroid')
        fig.savefig(spectral_centroid_path, bbox_inches='tight', pad_inches=0)
        plt.close(fig)
        features['spectral_centroid'] = spectral_centroid_path

        # 16. Spectral Bandwidth Plot
        fig, ax = compute_and_plot_spectral_bandwidth(y, sr)
        spectral_bandwidth_path = save_plot(fig, 'spectral_bandwidth')
        fig.savefig(spectral_bandwidth_path, bbox_inches='tight', pad_inches=0)
        plt.close(fig)
        features['spectral_bandwidth'] = spectral_bandwidth_path

        # 17. Power Spectrum Plot
        fig, ax = compute_and_plot_power_spectrum(y, sr)
        power_spectrum_path = save_plot(fig, 'power_spectrum')
        fig.savefig(power_spectrum_path, bbox_inches='tight', pad_inches=0)
        plt.close(fig)
        features['power_spectrum'] = power_spectrum_path

    except Exception as e:
        print(f"특징 추출 중 오류 발생: {file_path}\n오류 내용: {e}", file=sys.stderr)
        return None

    return {
        "file_path": file_path,
        "features": features
    }

# 플롯 함수들
def compute_and_plot_waveform(y: np.ndarray, sr: int):
    fig, ax = plt.subplots(figsize=(8, 2), dpi=100)  # 해상도 낮춤
    librosa.display.waveshow(y, sr=sr, ax=ax)
    ax.set_title('Waveform')
    ax.set_xlabel('Time (s)')
    ax.set_ylabel('Amplitude')
    plt.tight_layout()
    return fig, ax

def compute_and_plot_envelope(y: np.ndarray, sr: int):
    onset_env = librosa.onset.onset_strength(y=y, sr=sr)
    times = librosa.times_like(onset_env, sr=sr)
    fig, ax = plt.subplots(figsize=(8, 2), dpi=100)  # 해상도 낮춤
    ax.plot(times, onset_env)
    ax.set_title('Envelope')
    ax.set_xlabel('Time (s)')
    ax.set_ylabel('Amplitude')
    plt.tight_layout()
    return fig, ax

def compute_and_plot_cepstrum(y: np.ndarray, sr: int, min_freq: float, max_freq: float):
    spectrum = np.fft.fft(y)
    log_spectrum = np.log(np.abs(spectrum) + 1e-10)
    cepstrum = np.fft.ifft(log_spectrum).real
    quefrency = np.arange(len(cepstrum)) / sr
    idx = (quefrency >= 1 / max_freq) & (quefrency <= 1 / min_freq)
    fig, ax = plt.subplots(figsize=(8, 2), dpi=100)  # 해상도 낮춤
    ax.plot(quefrency[idx], cepstrum[idx])
    ax.set_title('Cepstrum')
    ax.set_xlabel('Quefrency (s)')
    ax.set_ylabel('Amplitude')
    plt.tight_layout()
    return fig, ax

def compute_and_plot_mel_spectrogram(y: np.ndarray, sr: int, display_axes: bool):
    S = librosa.feature.melspectrogram(y=y, sr=sr, fmin=20, fmax=8000)
    S_db = librosa.power_to_db(S, ref=np.max)
    fig, ax = plt.subplots(figsize=(8, 2), dpi=100)  # 해상도 낮춤
    img = librosa.display.specshow(S_db, sr=sr, fmin=20, fmax=8000, ax=ax, cmap='viridis')
    if not display_axes:
        ax.axis('off')
    else:
        ax.set_title('Mel Spectrogram')
        fig.colorbar(img, ax=ax, format='%+2.0f dB')
    plt.tight_layout(pad=0)
    return fig, ax

def compute_and_plot_linear_spectrogram(y: np.ndarray, sr: int, n_fft: int, hop_length: int, display_axes: bool):
    S = np.abs(librosa.stft(y, n_fft=n_fft, hop_length=hop_length))
    S_db = librosa.amplitude_to_db(S, ref=np.max)
    fig, ax = plt.subplots(figsize=(8, 2), dpi=100)  # 해상도 낮춤
    img = librosa.display.specshow(S_db, sr=sr, hop_length=hop_length, x_axis='time', y_axis='linear', ax=ax, cmap='magma')
    if not display_axes:
        ax.axis('off')
    else:
        ax.set_title('Linear Spectrogram')
        fig.colorbar(img, ax=ax, format='%+2.0f dB')
    plt.tight_layout(pad=0)
    return fig, ax

def compute_and_plot_mfcc(y: np.ndarray, sr: int, normalization: str = "min-max"):
    mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=20)
    if normalization == "min-max":
        mfccs_normalized = (mfccs - np.min(mfccs)) / (np.max(mfccs) - np.min(mfccs) + 1e-10)
    elif normalization == "z-score":
        mfccs_normalized = (mfccs - np.mean(mfccs)) / (np.std(mfccs) + 1e-10)
    else:
        raise ValueError("Unsupported normalization type. Use 'min-max' or 'z-score'.")
    fig, ax = plt.subplots(figsize=(8, 2), dpi=100)  # 해상도 낮춤
    img = librosa.display.specshow(mfccs_normalized, x_axis='time', ax=ax, cmap='coolwarm')
    ax.set_title('MFCC')
    fig.colorbar(img, ax=ax)
    plt.tight_layout(pad=0)
    return fig, ax

def compute_and_plot_chroma_features(y: np.ndarray, sr: int):
    chroma = librosa.feature.chroma_stft(y=y, sr=sr)
    fig, ax = plt.subplots(figsize=(8, 2), dpi=100)  # 해상도 낮춤
    img = librosa.display.specshow(chroma, x_axis='time', y_axis='chroma', cmap='coolwarm', ax=ax)
    ax.set_title('Chroma Features')
    fig.colorbar(img, ax=ax)
    plt.tight_layout(pad=0)
    return fig, ax

def compute_and_plot_spectral_centroid(y: np.ndarray, sr: int):
    spectral_centroids = librosa.feature.spectral_centroid(y=y, sr=sr)[0]
    times = librosa.times_like(spectral_centroids)
    fig, ax = plt.subplots(figsize=(8, 2), dpi=100)  # 해상도 낮춤
    ax.plot(times, spectral_centroids)
    ax.set_title('Spectral Centroid')
    ax.set_xlabel('Time (s)')
    ax.set_ylabel('Frequency (Hz)')
    plt.tight_layout()
    return fig, ax

def compute_and_plot_spectral_bandwidth(y: np.ndarray, sr: int):
    spectral_bandwidth = librosa.feature.spectral_bandwidth(y=y, sr=sr)[0]
    times = librosa.times_like(spectral_bandwidth)
    fig, ax = plt.subplots(figsize=(8, 2), dpi=100)  # 해상도 낮춤
    ax.plot(times, spectral_bandwidth)
    ax.set_title('Spectral Bandwidth')
    ax.set_xlabel('Time (s)')
    ax.set_ylabel('Bandwidth (Hz)')
    plt.tight_layout()
    return fig, ax

def compute_and_plot_power_spectrum(y: np.ndarray, sr: int):
    S = np.abs(np.fft.fft(y))**2
    freqs = np.fft.fftfreq(len(y), 1/sr)
    idx = np.argsort(freqs)
    idx = idx[freqs[idx] >= 0]
    freqs = freqs[idx]
    S = S[idx]
    fig, ax = plt.subplots(figsize=(8, 2), dpi=100)  # 해상도 낮춤
    ax.plot(freqs, S)
    ax.set_title('Power Spectrum')
    ax.set_xlabel('Frequency (Hz)')
    ax.set_ylabel('Power')
    plt.tight_layout()
    return fig, ax

def create_json_structure(data_entries, output_json_path):
    """
    Create a JSON file with structured data entries.

    Parameters:
        data_entries (list): List of feature dictionaries.
        output_json_path (str): Path to save the JSON file.
    """
    data = {"audio_features": data_entries}
    with open(output_json_path, 'w') as json_file:
        json.dump(data, json_file, indent=4)

def process_file(task):
    """
    Wrapper function to process a single file.

    Parameters:
        task (tuple): Tuple containing all necessary arguments.

    Returns:
        dict or None: Processed feature dictionary or None if failed.
    """
    class_name, data_type, file_path, result_folder, min_freq, max_freq, normalization = task
    features = extract_features(file_path, result_folder, min_freq, max_freq, normalization)
    if features is None:
        return None

    # 섹션 정보 추출 (파일명에서 'section_' 뒤의 두 글자)
    wav_file = os.path.basename(file_path)
    if 'section_' in wav_file:
        section_start = wav_file.find('section_') + len('section_')
        section = wav_file[section_start:section_start+2]
    else:
        section = '00'  # 기본값

    # note 추가 (자동 생성)
    note = f"This is a {class_name} machine, data type: {data_type}, file: {wav_file}"

    entry = {
        "file_path": features["file_path"],
        "machine_type": class_name,
        "section": section,
        "domain": data_type,  # 'train' 또는 'test'
        "features": features["features"],
        "note": note  # 주석 추가
    }
    return entry

def main():
    """
    Main function to process all files and extract features with progress tracking.
    """
    # 클래스 목록 및 데이터 타입 설정
    classes = ["ToyCar", "ToyTrain", "bearing", "fan", "gearbox", "slider", "valve"]
    data_types = ['test', 'train']
    all_entries = []

    # 데이터 디렉토리 설정
    base_dir = '../unziped/dev'

    # 모든 파일 목록 수집
    tasks = []
    for class_name in classes:
        for data_type in data_types:
            folder_path = os.path.join(base_dir, class_name, data_type)
            result_folder = os.path.join(folder_path, "Feature_Extraction_Results")
            os.makedirs(result_folder, exist_ok=True)

            if not os.path.exists(folder_path):
                print(f"폴더를 찾을 수 없습니다: {folder_path}", file=sys.stderr)
                continue

            # 모든 wav 파일 처리
            wav_files = [f for f in os.listdir(folder_path) if f.endswith('.wav')]
            for wav_file in wav_files:
                file_path = os.path.join(folder_path, wav_file)
                tasks.append((class_name, data_type, file_path, result_folder, 20, 8000, 'min-max'))

    # 디버깅: 총 작업 수 출력
    print(f"총 작업 수: {len(tasks)}")
    if len(tasks) == 0:
        print("처리할 파일이 없습니다. 데이터 경로와 파일 존재 여부를 확인해주세요.", file=sys.stderr)
        return

    # 멀티프로세싱 풀 생성 (CPU 코어 수에 맞춤)
    num_workers = max(1, cpu_count() - 1)  # 시스템 안정성을 위해 하나의 코어는 남겨둠
    print(f"사용 가능한 CPU 코어 수: {cpu_count()}, 사용되는 코어 수: {num_workers}")

    # 멀티프로세싱과 단순 진행 표시 (점 출력) 사용하여 병렬 처리 및 진행 상황 표시
    with Pool(processes=num_workers) as pool:
        for result in pool.imap_unordered(process_file, tasks):
            if result is not None:
                all_entries.append(result)
                # 점 출력 ('.') 후 플러시하여 즉시 표시
                print('.', end='', flush=True)

    # 줄바꿈 후 JSON 파일 저장 완료 메시지 출력
    print()  # 줄바꿈
    output_json_path = 'audio_features.json'
    create_json_structure(all_entries, output_json_path)
    print(f"JSON 파일이 '{output_json_path}' 경로에 저장되었습니다.")

# if __name__ == "__main__":
#     main()


총 작업 수: 8400
사용 가능한 CPU 코어 수: 12, 사용되는 코어 수: 11
.............................................................................................................

fan:   0%|          | 0/1200 [10:21<?, ?file/s]

.


gearbox:   0%|          | 0/1200 [10:23<?, ?file/s]

valve:   0%|          | 0/1200 [10:23<?, ?file/s]]


.

valve:   0%|          | 0/1200 [10:25<?, ?file/s]


........................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

TypeError: Object of type float32 is not JSON serializable

### 현재 위의 코드를 실행하면 png파일만 출력됨.

- 중간에 json에 경로나 특성을 저장하면서 오류가 발생함.
- png는 이미 다 생성됬으므로 json 경로와 실수 특성만 다시 생성하면 됨.


In [ ]:
import json
import os
import sys
import numpy as np
import librosa
import librosa.display
from multiprocessing import Pool, cpu_count

def extract_features(file_path, result_folder, min_freq=20, max_freq=8000, normalization='min-max'):
    """
    Extract numeric features from an audio file.

    Parameters:
        file_path (str): Path to the audio file.
        result_folder (str): Directory where existing feature plots are stored.
        min_freq (float): Minimum frequency for cepstrum plotting (unused).
        max_freq (float): Maximum frequency for cepstrum plotting (unused).
        normalization (str): Type of normalization for MFCC (unused).

    Returns:
        dict: Dictionary containing file path, numeric features, and plot paths.
    """
    try:
        # 오디오 로드
        y, sr = librosa.load(file_path, sr=None)
    except Exception as e:
        print(f"오디오 파일을 로드하는 중 오류 발생: {file_path}\n오류 내용: {e}", file=sys.stderr)
        return None

    # 특징 딕셔너리 초기화
    features = {}

    try:
        # 1. Zero Crossing Rate
        zero_crossing_rate = float(np.mean(librosa.feature.zero_crossing_rate(y=y)))
        features['zero_crossing_rate'] = zero_crossing_rate

        # 2. Harmonic to Noise Ratio (HNR)
        y_harmonic, y_percussive = librosa.effects.hpss(y)
        hnr = float(np.mean(np.abs(y_harmonic)) / (np.mean(np.abs(y_percussive)) + 1e-10))
        features['harmonic_to_noise_ratio'] = hnr

        # 3. Spectral Flatness
        spectral_flatness = float(np.mean(librosa.feature.spectral_flatness(y=y)))
        features['spectral_flatness'] = spectral_flatness

        # 4. Spectral Rolloff
        spectral_rolloff = float(np.mean(librosa.feature.spectral_rolloff(y=y, sr=sr)))
        features['spectral_rolloff'] = spectral_rolloff

        # 5. RMS Energy
        rms_energy = float(np.mean(librosa.feature.rms(y=y)))
        features['rms_energy'] = rms_energy

        # 6. Entropy (Shannon Entropy)
        histogram, bin_edges = np.histogram(y, bins=256, density=True)
        entropy = float(-np.sum(histogram * np.log2(histogram + 1e-10)))
        features['entropy'] = entropy

        # 7. Standard Deviation and Average
        features['std'] = float(np.std(y))
        features['avg'] = float(np.mean(y))

        # 이미지 경로 추가 (이미지가 이미 생성되어 있다고 가정)
        # Feature_Extraction_Results 폴더 내의 각 특징별 폴더에 이미지가 저장되어 있다고 가정
        plot_features = [
            'waveform',
            'envelope',
            'cepstrum',
            'mel_spectrogram_with_axes',
            'mel_spectrogram_no_axes',
            'mfcc',
            'linear_spectrogram_with_axes',
            'linear_spectrogram_no_axes',
            'chroma_features',
            'spectral_centroid',
            'spectral_bandwidth',
            'power_spectrum'
        ]

        for feature_name in plot_features:
            plot_folder = os.path.join(result_folder, feature_name)
            plot_path = os.path.join(plot_folder, f"{os.path.splitext(os.path.basename(file_path))[0]}.png")
            if os.path.exists(plot_path):
                features[feature_name] = plot_path
            else:
                features[feature_name] = None  # 또는 다른 적절한 값

    except Exception as e:
        print(f"특징 추출 중 오류 발생: {file_path}\n오류 내용: {e}", file=sys.stderr)
        return None

    return {
        "file_path": file_path,
        "features": features
    }

def create_json_structure(data_entries, output_json_path):
    """
    Create a JSON file with structured data entries.

    Parameters:
        data_entries (list): List of feature dictionaries.
        output_json_path (str): Path to save the JSON file.
    """
    data = {"audio_features": data_entries}
    with open(output_json_path, 'w') as json_file:
        json.dump(data, json_file, indent=4)

def process_file(task):
    """
    Wrapper function to process a single file.

    Parameters:
        task (tuple): Tuple containing all necessary arguments.

    Returns:
        dict or None: Processed feature dictionary or None if failed.
    """
    class_name, data_type, file_path, result_folder, min_freq, max_freq, normalization = task
    features = extract_features(file_path, result_folder, min_freq, max_freq, normalization)
    if features is None:
        return None

    # 섹션 정보 추출 (파일명에서 'section_' 뒤의 두 글자)
    wav_file = os.path.basename(file_path)
    if 'section_' in wav_file:
        section_start = wav_file.find('section_') + len('section_')
        section = wav_file[section_start:section_start+2]
    else:
        section = '00'  # 기본값

    # note 추가 (자동 생성)
    note = f"This is a {class_name} machine, data type: {data_type}, file: {wav_file}"

    entry = {
        "file_path": features["file_path"],
        "machine_type": class_name,
        "section": section,
        "domain": data_type,  # 'train' 또는 'test'
        "features": features["features"],
        "note": note  # 주석 추가
    }
    return entry

def main():
    """
    Main function to process all files and extract features with progress tracking.
    """
    # 클래스 목록 및 데이터 타입 설정
    classes = ["ToyCar", "ToyTrain", "bearing", "fan", "gearbox", "slider", "valve"]
    data_types = ['test', 'train']
    all_entries = []

    # 데이터 디렉토리 설정
    base_dir = '../unziped/dev'

    # 모든 파일 목록 수집
    tasks = []
    for class_name in classes:
        for data_type in data_types:
            folder_path = os.path.join(base_dir, class_name, data_type)
            result_folder = os.path.join(folder_path, "Feature_Extraction_Results")
            os.makedirs(result_folder, exist_ok=True)

            if not os.path.exists(folder_path):
                print(f"폴더를 찾을 수 없습니다: {folder_path}", file=sys.stderr)
                continue

            # 모든 wav 파일 처리
            wav_files = [f for f in os.listdir(folder_path) if f.endswith('.wav')]
            for wav_file in wav_files:
                file_path = os.path.join(folder_path, wav_file)
                tasks.append((class_name, data_type, file_path, result_folder, 20, 8000, 'min-max'))

    # 디버깅: 총 작업 수 출력
    print(f"총 작업 수: {len(tasks)}")
    if len(tasks) == 0:
        print("처리할 파일이 없습니다. 데이터 경로와 파일 존재 여부를 확인해주세요.", file=sys.stderr)
        return

    # 멀티프로세싱 풀 생성 (CPU 코어 수에 맞춤)
    num_workers = max(1, cpu_count() - 1)  # 시스템 안정성을 위해 하나의 코어는 남겨둠
    print(f"사용 가능한 CPU 코어 수: {cpu_count()}, 사용되는 코어 수: {num_workers}")

    # 멀티프로세싱과 단순 진행 표시 (점 출력) 사용하여 병렬 처리 및 진행 상황 표시
    with Pool(processes=num_workers) as pool:
        for result in pool.imap_unordered(process_file, tasks):
            if result is not None:
                all_entries.append(result)
                # 점 출력 ('.') 후 플러시하여 즉시 표시
                print('.', end='', flush=True)

    # 줄바꿈 후 JSON 파일 저장 완료 메시지 출력
    print()  # 줄바꿈
    output_json_path = 'audio_features.json'
    try:
        create_json_structure(all_entries, output_json_path)
        print(f"JSON 파일이 '{output_json_path}' 경로에 저장되었습니다.")
    except Exception as e:
        print(f"JSON 파일 저장 중 오류 발생: {e}", file=sys.stderr)

if __name__ == "__main__":
    main()
